In [44]:
import requests
standings_url = "https://www.pro-football-reference.com/years/2024/index.htm"
data = requests.get(standings_url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')
links = []
for table in standings_table:
    table_links = table.find_all('a')
    links += [l.get('href') for l in table_links if '/teams/' in l.get('href')]
team_urls = [f"https://www.pro-football-reference.com{l}" for l in links]
team_urls

[]

In [45]:
data = requests.get(team_urls[0])
import pandas as pd
from io import StringIO
games = pd.read_html(StringIO(data.text), match = "Schedule & Game Results")
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if l is not None and '/gamelog/' in l]
data = requests.get(f"https://www.pro-football-reference.com{links[0]}")
team_data = pd.read_html(StringIO(data.text), match="Regular Season")[0]
team_data.head()

IndexError: list index out of range

In [ ]:
years = list(range(2020,2023))
all_games = []
standings_url = "https://www.pro-football-reference.com/years/2024/index.htm"
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, "html.parser")
    standings_table = soup.select('table.stats_table')

    links = []
    for table in standings_table:
        table_links = table.find_all('a')
        links += [l.get('href') for l in table_links if '/teams/' in l.get('href')]
    team_urls = [f"https://www.pro-football-reference.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://www.pro-football-reference.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        games = pd.read_html(StringIO(data.text), match="Schedule & Game Results")
        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get('href') for l in links]
        links = [l for l in links if l is not None and '/gamelog/' in l]
        data = requests.get(r"https://www.pro-football-reference.com{links[0]}")
        team_data = pd.read_html(StringIO(data.text), match="Regular Season")[0]
        team_data.columns = team_data.columns.droplevel()

        team_data['Team'] = team_name
        team_data['Year'] = year
        all_games.append(team_data)
        time.sleep(1)  # To avoid overwhelming the server

IndexError: list index out of range